In [2]:
"""
This file contains our SQL Query to Wharton Research Dataservices (WRDS).

We connect to the database, and then submit a query per year. This code was
written by Keyi Wang, and is used to pull data from WRDS for the S&P 500 index
options data, S&P 500 index closing prices, and risk-free rate data.
"""

import time
from pathlib import Path
from dateutil.relativedelta import relativedelta
from datetime import datetime
import pandas as pd
import wrds

from settings import config

OUTPUT_DIR = Path(config("OUTPUT_DIR"))
DATA_DIR = Path(config("DATA_DIR"))
WRDS_USERNAME = config("WRDS_USERNAME")

In [ ]:


# # Read WRDS_USERNAME from .env
# WRDS_USERNAME = config("WRDS_USERNAME")

# # Connect to WRDS
# db = wrds.Connection(wrds_username=WRDS_USERNAME)

# # SECID for S&P 500 options
# secid = 108105  

# # Define the range of years
# years = range(1996, 2022)

# # Store query results
# option_data_list = []
# forward_price_list = []

# for year in years:
#     # --- Extract option data ---
#     option_query = f"""
#     SELECT date, secid, strike_price, cp_flag, best_bid, best_offer, volume, 
#            open_interest, impl_volatility, exdate
#     FROM optionm.opprcd{year}
#     WHERE secid = {secid} 
#     AND EXTRACT(DAY FROM date) = 15
#     """
#     option_df = db.raw_sql(option_query)
#     option_data_list.append(option_df)

# # Close WRDS connection
# db.close()




Loading library list...
Done


In [8]:
import wrds
import pandas as pd
from decouple import config

# Read WRDS_USERNAME from .env
WRDS_USERNAME = config("WRDS_USERNAME")

# Connect to WRDS
db = wrds.Connection(wrds_username=WRDS_USERNAME)

# SECID for S&P 500 options
secid = 108105  

# Define the range of years
years = range(1996, 2022)

# Store query results
option_data_list = []

for year in years:
    # --- Optimized query: support multiple expiration intervals ---
    option_query = f"""
    WITH first_trading_day AS (
        SELECT DATE_TRUNC('month', date) AS month, MIN(date) AS first_day
        FROM optionm.opprcd{year}
        WHERE secid = {secid}
        GROUP BY month
    )
    SELECT o.date, o.secid, o.strike_price, o.cp_flag, o.best_bid, o.best_offer, 
           o.volume, o.open_interest, o.impl_volatility, o.exdate
    FROM optionm.opprcd{year} o
    INNER JOIN first_trading_day f
    ON o.date = f.first_day
    WHERE o.secid = {secid}
    AND EXTRACT(MONTH FROM age(o.exdate, o.date)) IN (0, 1, 2, 3, 5, 6, 7, 10, 11, 12, 13);
    """
    
    option_df = db.raw_sql(option_query)   # days_to_expiration must be large than 7
    option_data_list.append(option_df)


# 关闭 WRDS 连接
db.close()



Loading library list...
Done


In [9]:
# Merge all data
option_data = pd.concat(option_data_list, ignore_index=True)
    
# Calculate mid price
option_data["mid_price"] = (option_data["best_bid"] + option_data["best_offer"]) / 2

# Convert date and exdate to datetime format
option_data["date"] = pd.to_datetime(option_data["date"])
option_data["exdate"] = pd.to_datetime(option_data["exdate"])

# Calculate the number of months between date and exdate
option_data["months_to_expiry"] = option_data.apply(
    lambda row: (relativedelta(row["exdate"], row["date"]).years * 12 + 
                 relativedelta(row["exdate"], row["date"]).months), 
    axis=1
)

# Sort data by date and months_to_expiry in ascending order
option_data = option_data.sort_values(by=["date", "months_to_expiry"]).reset_index(drop=True)

# Save data
option_data.to_csv("sp500_option_data_15th_sorted_1996_2021.csv", index=False)
print("✅ Data has been sorted by months_to_expiry and saved!")



✅ Data has been sorted by months_to_expiry and saved!


In [10]:
import pandas as pd
import numpy as np

# Load the SPX options data
df = pd.read_csv("sp500_option_data_15th_sorted_1996_2021.csv")

# ---- Filter 1: Drop options with special settlement ----

# ---- Filter 2: Select the quote with the highest open interest for duplicate quotes ----
df = df.sort_values(by=['date', 'exdate', 'cp_flag',  'strike_price'], ascending=[True, True, True, True])

# ---- Filter 3: Drop options with fewer than 7 days to maturity ----

# ---- Filter 4: Drop options with price less than 0.01 ----
df = df[df['mid_price'] >= 0.01]

# ---- Filter 5: Drop options with zero bid prices or negative bid-ask spreads ----
df = df[(df['best_bid'] > 0) & (df['best_offer'] > df['best_bid'])]

# ---- Filter 6: Drop options that violate static no-arbitrage bounds ----
# Ensure required columns exist

# ---- Filter 7: Use Existing Implied Volatility Column ----
df = df[(df['impl_volatility'] >= 0.05) & (df['impl_volatility'] <= 1.0)]

# ---- Save the cleaned data ----
df.to_csv("SPX_Options_Cleaned.csv", index=False)

print("✅ Cleaned SPX options data saved as 'SPX_Options_Cleaned.csv'.")
print(df.head())


✅ Cleaned SPX options data saved as 'SPX_Options_Cleaned.csv'.
          date     secid  strike_price cp_flag  best_bid  best_offer  volume  \
42  1996-01-04  108105.0      610000.0       C   10.0000      10.375   444.0   
20  1996-01-04  108105.0      615000.0       C    7.5000       8.000   465.0   
17  1996-01-04  108105.0      620000.0       C    4.5000       4.875  2606.0   
44  1996-01-04  108105.0      625000.0       C    2.2500       2.500  2671.0   
9   1996-01-04  108105.0      630000.0       C    1.1875       1.375  4022.0   

    open_interest  impl_volatility      exdate  mid_price  months_to_expiry  
42         5905.0         0.082711  1996-01-20   10.18750                 0  
20         4270.0         0.109019  1996-01-20    7.75000                 0  
17         6635.0         0.101986  1996-01-20    4.68750                 0  
44          252.0         0.094241  1996-01-20    2.37500                 0  
9          5969.0         0.097356  1996-01-20    1.28125         

In [11]:
import pandas as pd

# Read the CSV file
df = pd.read_csv("SPX_Options_Cleaned.csv")

# Ensure the 'date' column is in datetime format
df['date'] = pd.to_datetime(df['date'])

# Extract the year and month in "YYYY-MM" format
df['year_month'] = df['date'].dt.strftime('%Y-%m')

# Save the processed data to a new CSV file
df.to_csv("SPX_Options_Monthly.csv", index=False)

print(df.head())  # Display the first few rows to verify the transformation


        date     secid  strike_price cp_flag  best_bid  best_offer  volume  \
0 1996-01-04  108105.0      610000.0       C   10.0000      10.375   444.0   
1 1996-01-04  108105.0      615000.0       C    7.5000       8.000   465.0   
2 1996-01-04  108105.0      620000.0       C    4.5000       4.875  2606.0   
3 1996-01-04  108105.0      625000.0       C    2.2500       2.500  2671.0   
4 1996-01-04  108105.0      630000.0       C    1.1875       1.375  4022.0   

   open_interest  impl_volatility      exdate  mid_price  months_to_expiry  \
0         5905.0         0.082711  1996-01-20   10.18750                 0   
1         4270.0         0.109019  1996-01-20    7.75000                 0   
2         6635.0         0.101986  1996-01-20    4.68750                 0   
3          252.0         0.094241  1996-01-20    2.37500                 0   
4         5969.0         0.097356  1996-01-20    1.28125                 0   

  year_month  
0    1996-01  
1    1996-01  
2    1996-01  
3 

In [ ]:
import pandas as pd

# Read the CSV file
df = pd.read_csv("SP500_index_with_fwd_prices.csv")

# Ensure the 'date' column is in datetime format
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Extract the year and month in "YYYY-MM" format
df['year_month'] = df['date'].dt.strftime('%Y-%m')

# Save the processed data to a new CSV file
df.to_csv("SP500_index_monthly.csv", index=False)

# Display the first few rows to verify the transformation
print(df.head())



        date    sprtrn  spindx  risk free rate      fwd_6m    fwdprc_6m  \
0 1996-01-31  0.032617  636.02        0.071522  647.290526  647290.5261   
1 1996-02-29  0.006934  640.43        0.019370  649.158156  649158.1556   
2 1996-03-29  0.007917  645.50        0.026780  657.792004  657792.0039   
3 1996-04-30  0.013432  654.17        0.007053  665.488006  665488.0064   
4 1996-05-31  0.022853  669.12        0.044384  683.091414  683091.4137   

       Pt year_month  
0  636020    1996-01  
1  640430    1996-02  
2  645500    1996-03  
3  654170    1996-04  
4  669120    1996-05  


# Forward Price Calculation Formula

For a given time \( t \) with index price \( S_t \), the forward price for 6 months ahead, \( F_t(6) \), is calculated using the following formula:

\[
F_t(6) = S_t \times e^{r(6)}
\]

where:
- \( S_t \) is the spot index price at time \( t \);
- \( r(6) \) is the cumulative risk-free rate over the next 6 months, defined as:

  \[
  r(6) = \sum_{i=0}^{5} \frac{r_{t+i}}{12}
  \]

  where \( r_{t+i} \) represents the annualized risk-free rate at time \( t+i \), which is converted into a cumulative 6-month value.

## Calculation Steps
1. Read the index price \( S_t \) and the corresponding risk-free rate \( r_t \).
2. Compute the cumulative risk-free rate over the next 6 months:

   \[
   r(6) = \frac{1}{12} \sum_{i=0}^{5} r_{t+i}
   \]

3. Calculate the forward price:

   \[
   F_t(6) = S_t \times e^{r(6)}
   \]

4. If \( t+6 \) exceeds the data range, the forward price cannot be computed and is left as a null value.



In [3]:
# Connect to WRDS
db = wrds.Connection(wrds_username=WRDS_USERNAME)

# Store data for all years
rf_data_list = []

for year in range(1996, 2022):
    try:
        rf_query = f"""
        SELECT date, borrowrate
        FROM optionm.borrate{year}
        WHERE borrowrate != -99.990000  -- Filter out invalid data
        """
        rf_year_data = db.raw_sql(rf_query)

        if not rf_year_data.empty:
            rf_data_list.append(rf_year_data)
        else:
            print(f"⚠️ No valid Borrow Rate data for {year}.")

    except Exception as e:
        print(f"❌ Failed to retrieve `borrate` data for {year}: {e}")

# Merge data from all years
if rf_data_list:
    rf_data = pd.concat(rf_data_list, ignore_index=True)
else:
    print("❌ No Borrow Rate data retrieved!")

# Close WRDS connection
db.close()


Loading library list...
Done


In [ ]:
# Select the first record of each month
monthly_rf_data = rf_data.resample('M').first().reset_index()

# Ensure no invalid data is included
monthly_rf_data = monthly_rf_data[monthly_rf_data["borrowrate"] != -99.990000]

# Save data to a CSV file
monthly_rf_data.to_csv("monthly_risk_free_rate.csv", index=False)

print("✅ Monthly risk-free rate has been saved to `monthly_risk_free_rate.csv`!")
# Display results
print(monthly_rf_data.head())



C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_25992\1887038002.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_rf_data = rf_data.resample('M').first().reset_index()


✅ 每月的风险利率已保存至 `monthly_risk_free_rate.csv`！
        date  borrowrate
0 1996-01-31    0.071522
1 1996-02-29    0.019370
2 1996-03-31    0.026780
3 1996-04-30    0.007053
4 1996-05-31    0.044384


In [ ]:
conn = wrds.Connection(wrds_username='WRDS_USERNAME')  # Connect to WRDS

# Query the S&P 500 index options data
query = """
SELECT caldt, sprtrn, spindx
FROM crsp.msp500
WHERE caldt BETWEEN '1996-01-01' AND '2021-12-31'
"""
df_index = conn.raw_sql(query)

# Rename and format the date column
df_index = df_index.rename(columns={'caldt': 'date'})
df_index['date'] = pd.to_datetime(df_index['date'])

# Save as CSV
df_index.to_csv("SP500_index_data_1996_2021.csv", index=False)

# Check output
print("CSV file saved: SP500_index_data_1996_2021.csv")
print(df_index.head())

# Close WRDS connection
conn.close()



WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done
CSV file saved: SP500_index_data_1996_2021.csv
        date    sprtrn  spindx
0 1996-01-31  0.032617  636.02
1 1996-02-29  0.006934  640.43
2 1996-03-29  0.007917  645.50
3 1996-04-30  0.013432  654.17
4 1996-05-31  0.022853  669.12
